#### We need to install the ktrain library. Its a light weight wrapper for keras to help train neural networks. With only a few lines of code it allows you to build models, estimate optimal learning rate, loading and preprocessing text and image data from various sources and much more. More about our approach can be found at [this](https://towardsdatascience.com/bert-text-classification-in-3-lines-of-code-using-keras-264db7e7a358) article.

In [1]:
# To install only the requirements of this notebook, uncomment the lines below and run this cell

# ===========================

!pip install numpy==1.19.5
!pip install pandas==1.1.5
!pip install ktrain==0.26.3

# ===========================

     |████████████████████████████████| 25.3MB 1.6MB/s 
     |████████████████████████████████| 6.8MB 41.4MB/s 
     |████████████████████████████████| 983kB 43.3MB/s 
     |████████████████████████████████| 266kB 50.8MB/s 
     |████████████████████████████████| 1.9MB 37.5MB/s 
     |████████████████████████████████| 1.2MB 41.8MB/s 
     |████████████████████████████████| 471kB 34.8MB/s 
     |████████████████████████████████| 901kB 41.4MB/s 
     |████████████████████████████████| 3.3MB 37.9MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.3-cp37-none-any.whl size=25282390 sha256=0f129e50aaa4d78ab674e5f6b95d1c66df8f4fa6b62a1ac02b1867c70bbbdecd
  Stored in directory: /root/.cache/pip/wheels/16/05/be/d6e659b3349016b1059e19fa028f165af4eeae2c196f329112
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993242 sha256=489499c000ae032ae91b31fdedc7ec2d0ef1fccbd3b997508abed818cd45e520
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7d

In [2]:
# To install the requirements for the entire chapter, uncomment the lines below and run this cell

# ===========================

# try:
#     import google.colab
#     !curl  https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/ch4-requirements.txt | xargs -n 1 -L 1 pip install
# except ModuleNotFoundError:
#     !pip install -r "ch4-requirements.txt"

# ===========================

In [3]:
# use tensorflow 2.4.0 for this notebook
!pip install tensorflow==2.4.0

     |████████████████████████████████| 394.7MB 41kB/s 
     |████████████████████████████████| 2.9MB 45.8MB/s 
     |████████████████████████████████| 471kB 39.7MB/s 
     |████████████████████████████████| 3.8MB 37.5MB/s 
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Found existing installation: grpcio 1.34.1
    Uninstalling grpcio-1.34.1:
      Successfully uninstalled grpcio-1.34.1
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:
      Successfully uninstalled tensorflow-2.5.0


In [4]:
#Importing
import ktrain
from ktrain import text

In [5]:
##obtain the dataset
import os
try :
    from google.colab import files
    import tensorflow as tf
    dataset = tf.keras.utils.get_file(
        fname="aclImdb.tar.gz", 
        origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
        extract=True,
    )
    IMDB_DATADIR = os.path.join(os.path.dirname(dataset), "aclImdb")
except ModuleNotFoundError :
    if not os.path.exists(os.getcwd()+"\\Data\\aclImdb") :
        import tensorflow as tf
        dataset = tf.keras.utils.get_file(
            fname="aclImdb.tar.gz", 
            origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
            extract=True,
        )

        # set path to dataset
        IMDB_DATADIR=os.getcwd()
    else :

        # set path to dataset
        IMDB_DATADIR=os.getcwd()+"\\Data\\aclImdb"

84131840/84125825 [==============================] - 6s 0us/step


## STEP 1: Preprocessing
The texts_from_folder function will load the training and validation data from the specified folder and automatically preprocess it according to BERT's requirements. In doing so, the BERT model and vocabulary will be automatically downloaded.

In [6]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder(IMDB_DATADIR, 
                                                                       maxlen=500, 
                                                                       preprocess_mode='bert',
                                                                       train_test_names=['train', 
                                                                                         'test'],
                                                                       classes=['pos', 'neg'])

detected encoding: utf-8
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


### STEP 2: Loading a pre trained BERT and wrapping it in a ktrain.learner object

In [7]:
model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model,train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=6)

Is Multi-Label? False
maxlen is 500
done.


### STEP 3: Training and Tuning the model's parameters

In [8]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
4167/4167 [==============================] - 2358s 561ms/step - loss: 0.3327 - accuracy: 0.8504 - val_loss: 0.1843 - val_accuracy: 0.9311
Epoch 2/4
4167/4167 [==============================] - 2325s 558ms/step - loss: 0.1542 - accuracy: 0.9423 - val_loss: 0.2223 - val_accuracy: 0.9138
Epoch 3/4
4167/4167 [==============================] - 2323s 557ms/step - loss: 0.0899 - accuracy: 0.9677 - val_loss: 0.1847 - val_accuracy: 0.9350
Epoch 4/4
4167/4167 [==============================] - 2322s 557ms/step - loss: 0.0247 - accuracy: 0.9934 - val_loss: 0.2330 - val_accuracy: 0.9416
